# WML-A Job Submission via WML-A CLI

Offical examples can be found here: https://wmla-console-cpd-wmla.apps.cpd.mskcc.org/ui/#/cliTools

In [1]:
%env DIR=/userfs
%env NAMESPACE=cpd
%env HOST=wmla-console-cpd.apps.cpd.mskcc.org
%env BASE_URL=https://cpd-cpd.apps.cpd.mskcc.org

%env dlicmd=wmla-utils/dlicmd.py

%env VOLUME_DISPLAY_NAME=cpd::demo-project-pvc

env: DIR=/userfs
env: NAMESPACE=cpd
env: HOST=wmla-console-cpd.apps.cpd.mskcc.org
env: BASE_URL=https://cpd-cpd.apps.cpd.mskcc.org
env: dlicmd=wmla-utils/dlicmd.py
env: VOLUME_DISPLAY_NAME=cpd::demo-project-pvc


### Submit Jobs
#### PyTorch (single GPU or multiple GPUs on one node with multithreading)

In [2]:
%env DIR_job_submission=/userfs/job_submission
%env file_exec=train_command.py

!rm -rf $DIR_job_submission
!mkdir -p $DIR_job_submission

!cp -r $DIR/deepliif-repo/deepliif $DIR_job_submission
!cp $DIR/deepliif-repo/* $DIR_job_submission
!cp $DIR/$file_exec $DIR_job_submission
!cp $DIR/monitor_gpu.sh $DIR_job_submission
!cp $DIR/download_data.sh $DIR_job_submission
!cp $DIR/storage_volume_utils.py $DIR_job_submission
!cp $DIR/cpd_utils.py $DIR_job_submission

env: DIR_job_submission=/userfs/job_submission
env: file_exec=train_command.py
cp: -r not specified; omitting directory '/userfs/deepliif-repo/Datasets'
cp: -r not specified; omitting directory '/userfs/deepliif-repo/deepliif'
cp: -r not specified; omitting directory '/userfs/deepliif-repo/DeepLIIF_Statistics'
cp: -r not specified; omitting directory '/userfs/deepliif-repo/docs'
cp: -r not specified; omitting directory '/userfs/deepliif-repo/images'
cp: -r not specified; omitting directory '/userfs/deepliif-repo/model-server'
cp: -r not specified; omitting directory '/userfs/deepliif-repo/Registration'
cp: -r not specified; omitting directory '/userfs/deepliif-repo/Sample_Large_Tissues'
cp: -r not specified; omitting directory '/userfs/deepliif-repo/Scripts'


In [350]:
!python $dlicmd --exec-start PyTorch --rest-host $HOST --rest-port -1 --jwt-token $USER_ACCESS_TOKEN \
                  --msd-env USER_ACCESS_TOKEN=$USER_ACCESS_TOKEN --msd-env BASE_URL=$BASE_URL \
                  --msd-env VOLUME_DISPLAY_NAME=$VOLUME_DISPLAY_NAME \
                  --workerDeviceNum 1 --workerMemory 8g \
                  --model-dir $DIR_job_submission --model-main $file_exec \
                  --cs-datastore-meta type=fs,data_path=DeepLIIF_Datasets/

Copying files and directories ...
Content size: 64.2K
{
  "execId": "cpd-77",
  "appId": "cpd-77"
}


### Submit Jobs
#### distPyTorch (multiprocessing using DDP)

In [400]:
%env DIR_job_submission=/userfs/job_submission
%env file_exec=train_command.py

!rm -rf $DIR_job_submission
!mkdir -p $DIR_job_submission

!cp -r $DIR/deepliif-repo/deepliif $DIR_job_submission
!cp $DIR/deepliif-repo/* $DIR_job_submission
!cp $DIR/$file_exec $DIR_job_submission
!cp $DIR/monitor_gpu.sh $DIR_job_submission
!cp $DIR/download_data.sh $DIR_job_submission
!cp $DIR/storage_volume_utils.py $DIR_job_submission
!cp $DIR/cpd_utils.py $DIR_job_submission

env: DIR_job_submission=/userfs/job_submission
env: file_exec=train_command.py
cp: -r not specified; omitting directory '/userfs/deepliif-repo/Datasets'
cp: -r not specified; omitting directory '/userfs/deepliif-repo/deepliif'
cp: -r not specified; omitting directory '/userfs/deepliif-repo/DeepLIIF_Statistics'
cp: -r not specified; omitting directory '/userfs/deepliif-repo/docs'
cp: -r not specified; omitting directory '/userfs/deepliif-repo/images'
cp: -r not specified; omitting directory '/userfs/deepliif-repo/model-server'
cp: -r not specified; omitting directory '/userfs/deepliif-repo/Registration'
cp: -r not specified; omitting directory '/userfs/deepliif-repo/Sample_Large_Tissues'
cp: -r not specified; omitting directory '/userfs/deepliif-repo/Scripts'


In [401]:
%%writefile /userfs/conf_distPyTorch.py
import os
import torch.distributed as dist
def init_process():
    dist.init_process_group(
        backend='nccl',
        init_method='tcp://' + os.environ['MASTER_ADDR'] + ':' + os.environ['MASTER_PORT'],
        rank=int(os.environ['RANK']),
        world_size=int(os.environ['WORLD_SIZE']))
    
print('------ initiate process group... ------')
init_process()

Overwriting /userfs/conf_distPyTorch.py


In [402]:
# cat cannot change file in place, so we create a new one and use it to overwrite cli.py
!cat /userfs/conf_distPyTorch.py $DIR_job_submission/cli.py > $DIR_job_submission/cli_edited.py
!mv $DIR_job_submission/cli_edited.py $DIR_job_submission/cli.py

In [411]:
!python $dlicmd --exec-start distPyTorch --rest-host $HOST --rest-port -1 --jwt-token $USER_ACCESS_TOKEN \
                  --msd-env USER_ACCESS_TOKEN=$USER_ACCESS_TOKEN --msd-env BASE_URL=$BASE_URL \
                  --msd-env VOLUME_DISPLAY_NAME=$VOLUME_DISPLAY_NAME \
                  --numWorker 6 --workerMemory 8g \
                  --model-dir $DIR_job_submission --model-main $file_exec \
                  --data-source '[{"type":"fs", "location":{"volume":"cpd::demo-project-pvc"}}]' \
                  --data-source '[{"type":"fs", "location":{"volume":"DeepLIIFData"}}]'

Copying files and directories ...
Content size: 64.2K
{
  "execId": "cpd-83",
  "appId": "cpd-83"
}


In [404]:
data_source = '[{"type": "connection", "asset": {"asset_id": "ea76a8f1-eab8-4a00-8bf3-ce31ade3fdc4", "project_id": "e008ff36-c41d-4f57-968f-639f9b5bb229"}, "location": {"paths": "t10k-labels-idx1-ubyte.gz, model-wl5zj11q/training-output.json", "bucket": "cos-demo-more"}}]’

SyntaxError: unterminated string literal (detected at line 1) (4148379961.py, line 1)

#### Specify Job/App ID

In [412]:
%env APP_ID=cpd-83

env: APP_ID=cpd-83


### Delete Jobs (and associated results/logs)
#### delete one job

In [395]:
# !python $dlicmd.py --exec-delete $NAMESPACE-76 --rest-host $HOST --rest-port -1 --jwt-token $USER_ACCESS_TOKEN 

#### delete multiple jobs in a loop

In [4]:
# for i in range(70, 99):
#     %env i=$i
#     !python $dlicmd --exec-delete $NAMESPACE-$i --rest-host $HOST --rest-port -1 --jwt-token $USER_ACCESS_TOKEN 

env: i=70
Usage:
   python dlicmd.py --help
   python dlicmd.py --logon <connection-options> <logon-options>
   python dlicmd.py --dl-frameworks | --exec-get-all | --exec-delete-all <connection-options>
   python dlicmd.py --exec-start <framework-name> <connection-options> <datastore-meta> <submit-arguments>
   python dlicmd.py --exec-get | --exec-stop | --exec-delete <exec-id> <connection-options>
   python dlicmd.py --exec-deploy <connection-options> <deploy-arguments>
   python dlicmd.py --exec-launcherlogs | --exec-outlogs | --exec-errlogs <exec-id> <connection-options>
   python dlicmd.py --exec-trainlogs | --exec-trainoutlogs |--exec-trainerrlogs | --exec-trainresult <exec-id> <connection-options>

Commands:
   --help               Print help message.
   --logon              Logs the user onto IBM Watson Machine Learning Accelerator
   --dl-frameworks      List available deep learning frameworks for exec.
   --exec-start         Submit a deep learning exec.
   --exec-get-all     

### Get Job Status

In [413]:
!python $dlicmd --exec-get $APP_ID --rest-host $HOST --rest-port -1 --jwt-token $USER_ACCESS_TOKEN

{
  "id": "cpd-83",
  "args": "--exec-start distPyTorch --msd-env USER_ACCESS_TOKEN=eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6IjNMVWFNdUp3UE1nNkhpaTZZQTZrTWhfNWxJYTZlQ0hpbkNtejNzXzBOYUUifQ.eyJ1aWQiOiIxMDAwMzMxMDg5IiwidXNlcm5hbWUiOiJraGFybGFkIiwicm9sZSI6IlVzZXIiLCJwZXJtaXNzaW9ucyI6WyJjcmVhdGVfcHJvamVjdCIsImFjY2Vzc19jYXRhbG9nIiwiYWNjZXNzX2luZm9ybWF0aW9uX2Fzc2V0cyIsInZpZXdfcXVhbGl0eSIsImNyZWF0ZV9zcGFjZSIsIm1hbmFnZV9pbmZvcm1hdGlvbl9hc3NldHMiLCJtYW5hZ2VfbWV0YWRhdGFfaW1wb3J0IiwibWFuYWdlX2Rpc2NvdmVyeSIsIm1hbmFnZV9xdWFsaXR5Iiwidmlld19nb3Zlcm5hbmNlX2FydGlmYWN0cyIsImF1dGhvcl9nb3Zlcm5hbmNlX2FydGlmYWN0cyIsImNhbl9wcm92aXNpb24iLCJzaWduX2luX29ubHkiXSwiZ3JvdXBzIjpbMTAwMDBdLCJzdWIiOiJraGFybGFkIiwiaXNzIjoiS05PWFNTTyIsImF1ZCI6IkRTWCIsImlhdCI6MTY5ODE2NTg1MSwiZXhwIjo1Mjk4MTYyMjUxfQ.jgz8NgOUTCnCdUFS_iR2TuPkAQMCV0c1yPIAhQ783Zl61U5sM0zU65SE-_8wpCCjgdlRdjawTir22Eht-rMdO8M7vfXm84BgBhtEt-408DzB4csMPrwweGZ23yTIugdEeDRtuVOdvY-EeCvNqAk4HDaiKntwQGdEq0EARSz7vCo6cZjipyuxDwdct7v4WmKgVYh-HGvwCXrYZlRYNyUOO6rHPYkPv2JDXN

### Get Job Log
#### last 10 lines

In [370]:
# !python $dlicmd --exec-outlogs $APP_ID --rest-host $HOST --rest-port -1 --jwt-token $USER_ACCESS_TOKEN

#### full log files

In [396]:
!python $dlicmd --exec-trainerrlogs $APP_ID --rest-host $HOST --rest-port -1 --jwt-token $USER_ACCESS_TOKEN

In [414]:
!python $dlicmd --exec-trainoutlogs $APP_ID --rest-host $HOST --rest-port -1 --jwt-token $USER_ACCESS_TOKEN

['inference.log', 'edi_deployments', 'images', 'DeepLIIF_Datasets', 'openscale_metrics', 'deepliif-ws-output', 'model_output_2022-02-08', 'checkpoints']
['MNIST', 'deepliif', 'DeepLIIF_Datasets']
['mydatafs', 'mygpfs', 'myresultfs']
[]
['Multi-GPU Training.md', 'setup.py', 'monitor_gpu.sh', 'cli.py', 'LICENSE.md', 'download_data.sh', 'cpd_utils.py', 'environment.yml', 'deepliif', 'Dockerfile', 'PostProcessSegmentationMask.py', 'train_command.py', 'README.md', 'storage_volume_utils.py']
/mnts/DeepLIIFData
['inference.log', 'edi_deployments', 'images', 'DeepLIIF_Datasets', 'openscale_metrics', 'deepliif-ws-output', 'model_output_2022-02-08', 'checkpoints']
/gpfs/myresultfs/kharlad/batchworkdir/cpd-83/_submitted_code/job_submission
['MNIST', 'raw']
['bin', 'boot', 'dev', 'etc', 'home', 'lib', 'lib64', 'lost+found', 'media', 'mnt', 'opt', 'proc', 'root', 'run', 'sbin', 'srv', 'sys', 'tmp', 'usr', 'var', 'wmla-logging', 'mnts', 'gpfs', 'licenses']
['DeepLIIFData']
['inference.log', 'edi_dep

## Others - Visualizer Application

In [ ]:
from ws_applications import display_link

In [ ]:
display_link()